In [9]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer  # For TF-IDF feature extraction
from sklearn.naive_bayes import MultinomialNB  # Naive Bayes classifier
from sklearn.model_selection import train_test_split  # Splitting data
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import random
import nltk
import numpy as np


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/shreyaschandrashekar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shreyaschandrashekar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [17]:

import json
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
dataset_path="data.json"
with open(dataset_path, 'r') as f:
  data = json.load(f)
def preprocess_data(dataset_path):
  """
  Preprocesses chatbot intent data from a JSON file.

  Args:
      dataset_path (str): Path to the JSON file containing intents data.

  Returns:
      tuple: A tuple containing the following elements:
          - words (list): List of all cleaned and lemmatized words.
          - classes (list): List of unique intent tags.
          - documents (list): List of tuples containing (cleaned message, intent tag).
          - ignore_words (list): List of words to ignore during pre-processing (optional).
  """

  words = []
  classes = []
  documents = []
  ignore_words = ['?', '!']
  lemmatizer = WordNetLemmatizer()

  # Load the JSON data
  with open(dataset_path, 'r') as f:
    data = json.load(f)

  # Iterate through intents and patterns
  for intent in data["intents"]:
    for pattern in intent['patterns']:
      # Tokenize the pattern (word separation)
      w = nltk.word_tokenize(pattern.lower())
      words.extend(w)

      # Add document (cleaned message, intent tag)
      cleaned_pattern = [lemmatizer.lemmatize(word) for word in w if word not in ignore_words]
      documents.append((cleaned_pattern, intent['tag']))

      # Add classes (unique intent tags)
      if intent['tag'] not in classes:
        classes.append(intent['tag'])

  # Sort and remove duplicates from words and classes
  words = sorted(list(set(words)))
  classes = sorted(list(set(classes)))

  return words, classes, documents, ignore_words

# Example usage
dataset_path = "data.json"  # Replace with your actual dataset path
words, classes, documents, ignore_words = preprocess_data(dataset_path)

# Print some information (optional)
print("Number of words:", len(words))
print("Number of classes (intent tags):", len(classes))
print("Sample document:", documents[0])

Number of words: 131
Number of classes (intent tags): 27
Sample document: (['hi', 'there'], 'greetings')


In [4]:
print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


80 documents
27 classes ['advancement of ayurveda', 'arthritis', 'ayurbot', 'ayurveda and homeopathy', 'ayurvedic medication', 'body pain', 'cancer', 'cold', 'cough', 'diabetes', 'drugs and its uses', 'fever', 'goodbye', 'greetings', 'headache', 'heart disease', 'history', 'joints pain', 'pregnancy', 'red eyes', 'running nose', 'side effects of ayurveda', 'sour throat', 'stomachache', 'thanks', 'wet cough', 'why ayurveda']
131 unique lemmatized words ['&', "'s", '(', ')', ',', '-', ':', 'a', 'about', 'advancement', 'advantage', 'all', 'and', 'anjana', 'application', 'are', 'aristab', 'arka', 'arthritis', 'asava', 'avaleha', 'ayurveda', 'ayurvedic', 'bad', 'better', 'between', 'bhasma', 'bodypain', 'bye', 'c', 'cancer', 'chuma', 'churna', 'churna/kashayam', 'cold', 'cough', 'd', 'describe', 'diabetes', 'difference', 'disadvantage', 'disease', 'drug', 'dry', 'e', 'effect', 'eye', 'f', 'for', 'g', 'ghrita', 'goodbye', 'guggulu', 'gutika', 'h', 'have', 'headache', 'heart', 'hello', 'hi', '

In [5]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [18]:
def predict_class(model, vectorizer, message):
  """
  Predicts the intent class for a given user message.

  Args:
      model (sklearn.naive_bayes.MultinomialNB): Trained Naive Bayes model.
      vectorizer (sklearn.feature_extraction.text.CountVectorizer): Fitted vectorizer.
      message (str): User message to predict the intent for.

  Returns:
      str: The predicted intent class (tag).
  """

  # Preprocess the message
  words = nltk.word_tokenize(message.lower())
  cleaned_pattern = [lemmatizer.lemmatize(word) for word in words if word not in ignore_words]
  # Convert the message to a vector
  new_vector = vectorizer.transform([cleaned_pattern])
  # Predict the class
  predicted_class = model.predict(new_vector)[0]
  return predicted_class

# Example usage
dataset_path = "data.json"  # Replace with your actual dataset path
words, classes, documents, ignore_words = preprocess_data(dataset_path)

# Feature extraction and model training
vectorizer = CountVectorizer()
X = vectorizer.fit_transform([doc[0] for doc in documents])  # Get words from documents
labels = [intent for _, intent in documents]  # Get intent tags from documents
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

classifier = MultinomialNB()
classifier.fit(X_train, y_train)
# Optional: Evaluate the model on the testing data (accuracy)

# Chatbot interaction loop
lemmatizer = WordNetLemmatizer()  # Needed for real-time message pre-processing

while True:
  message = input("You: ")
  # Preprocess the message
  words = nltk.word_tokenize(message.lower())
  cleaned_pattern = [lemmatizer.lemmatize(word) for word in words if word not in ignore_words]
  # Convert the message to a vector
  new_vector = vectorizer.transform([cleaned_pattern])
  # Predict the class
  predicted_class = classifier.predict(new_vector)[0]

  # Respond based on the predicted class (replace with your actual responses)
  print("Bot:", greetings[predicted_class] if predicted_class in greetings else "Sorry, I don't understand.")

# Example greetings dictionary (replace with your responses for each intent)
greetings = {
  "greeting": "Hi there!",
  "goodbye": "Goodbye! Have a nice day.",
  # ... (add more greetings for different intents)
}

AttributeError: 'list' object has no attribute 'lower'

In [22]:
from sklearn.feature_extraction.text import CountVectorizer  # Using CountVectorizer for this example
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

# Create a CountVectorizer (alternative: TF-IDFVectorizer)
vectorizer = CountVectorizer()
empty_docs = [doc for doc in documents if not doc[0]]  # Check if any document's word list (first element) is empty


In [27]:
print(len(empty_docs))

0


In [ ]:
if empty_docs:
  print("Warning: Found empty documents after pre-processing:", len(empty_docs))

vectorizer = CountVectorizer()

# Attempt to fit the vectorizer
try:
  X = vectorizer.fit_transform([[word.lower() for word in doc[0]] for doc in documents])
except ValueError as e:
  if "empty vocabulary" in str(e):
    print("Error: Empty vocabulary. Consider reviewing pre-processing or using TF-IDF.")
  else:
    raise e 
# Separate features and labels
labels = [doc[1] for doc in documents]  # Get intent tags using indexing

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)  # Split data 80% train, 20% test

# Train the Naive Bayes classifier
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

# Compile the model (specifying loss function, optimizer, and metrics)
classifier.compile(loss='multinomial_nb', optimizer='adam', metrics=['accuracy'])

# Train the model on the training data (already done in fit)
# classifier.fit(X_train, y_train)  # This line is already executed in the previous step

# Evaluate the model on the testing data (optional)
# results = classifier.evaluate(X_test, y_test)
# print("Test Loss:", results[0])
# print("Test Accuracy:", results[1])
